# Model building

In [17]:
# Imports 

import requests
import zipfile
import os
import pandas as pd
import ftplib
import os
from getpass import getpass
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data/preprocessed_data.csv')

In [3]:
data

,Unnamed: 0,EstimatedTime,LAT,LON,Heading,SOG_norm,COG_norm,GrossTonnage_norm,VHM0_norm,VMDR_norm,Temperature_norm,Salinity_norm
0,0,2020-01-01 00:00:00,36.80096,-75.22302,283.0,0.309783,0.185836,0.360360,0.216849,0.278556,0.375870,0.887591
1,1,2020-01-01 00:00:00,26.08420,-79.48273,187.0,0.331522,0.961661,0.515015,0.088924,0.052832,0.835220,0.982673
2,2,2020-01-01 00:00:00,29.34368,-94.74366,86.0,0.168478,0.709402,0.293093,0.018721,0.323412,0.490509,0.792078
3,3,2020-01-01 00:00:00,37.76973,-122.35415,326.0,0.190217,0.286447,0.293093,0.084243,0.745706,0.377345,0.641309
4,4,2020-01-01 00:00:00,26.89363,-79.20262,155.0,0.684783,0.878388,0.451652,0.092044,0.022122,0.808156,0.987242
...,...,...,...,...,...,...,...,...,...,...,...,...
566841,720152,2020-10-02 00:00:00,26.05327,-79.84955,5.0,0.684783,0.510379,0.371572,0.107644,0.056417,0.944354,0.973406
566842,720153,2020-10-02 00:00:00,28.44555,-95.59912,57.0,0.315217,0.636874,0.479279,0.037441,0.305431,0.861402,0.840567
566843,720154,2020-10-02 00:00:00,39.18945,-130.91286,280.0,0.695652,0.183639,0.706306,0.404056,0.767189,0.674095,0.874057
566844,720155,2020-10-02 00:00:00,17.18697,-67.33857,110.0,0.250000,0.768742,0.302703,0.371295,0.258602,0.934577,0.935218


## Simple model: linear regression

In [21]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [15]:
# target 
y = data.SOG_norm

# predictors
features = ['Heading','COG_norm','GrossTonnage_norm','VHM0_norm','VMDR_norm','Temperature_norm','Salinity_norm'] 
X = data[features]

# split data to train and validation
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

In [19]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_X, train_y)

# Make predictions using the testing set
pred_y = regr.predict(val_X)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(val_y, pred_y))
# R2: 1 is perfect prediction
print('R^2: %.2f'
      % r2_score(val_y, pred_y))

Coefficients: 
 [-6.67362977e-05  1.21379688e-02  2.73277047e-01  5.25732675e-02
  2.92452506e-02  1.23396015e-01  4.43650989e-02]
Mean squared error: 0.02
Coefficient of determination: 0.17


## Random forest

In [22]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
pred_y = forest_model.predict(val_X)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(val_y, pred_y))
# R2: 1 is perfect prediction
print('R^2: %.2f'
      % r2_score(val_y, pred_y))

Mean squared error: 0.00
R^2: 0.98
